## Functional API intro

In [34]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense,Dropout,Input,Flatten
from tensorflow.python.keras.utils.vis_utils import plot_model
from keras_visualizer import visualizer

In [37]:
def build_model_with_functional():
    input_layer = Input(shape = (28,28))
    flatten_layer = Flatten()(input_layer)
    first_dense = Dense(128,activation='relu')(flatten_layer)
    output_layer = Dense(10,activation='softmax')(first_dense)
    
    model = tf.keras.models.Model(inputs=input_layer,outputs=output_layer)
    return model 

In [38]:
model = build_model_with_functional()
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


## Testing this functional_api model on fashion_mnist

In [39]:
from tensorflow.keras.datasets import fashion_mnist
mnist = tf.keras.datasets.fashion_mnist
(x_train ,y_train),(x_test,y_test) = mnist.load_data()
x_train = x_train/255.0
x_test = x_test/255.0

In [41]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=5)
model.evaluate(x_test,y_test)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1675 - accuracy: 0.9365
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1636 - accuracy: 0.9386
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1601 - accuracy: 0.9394
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1569 - accuracy: 0.9409
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3915 - accuracy: 0.8884


[0.3915116488933563, 0.8884000182151794]

# Energy efficiency dataset : multi-output model

In [45]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [43]:
URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx'
df = pd.read_excel(URL)
print(df.head)

NameError: name 'pd' is not defined